## Notebook For Eclipse Software Deployment ##
<i>Written by David Story </i>

___
### Description
This notebook was written to be an easy-to-use plateform for deploying and implementing software to AWS cloud computing services and creating datasets for the Eclipse Megamovie project, as well as related computationally heavy research projects.

This notebook will automate a number of tasks for preparing your AWS EC2 instances and deploying software to those instances. 

This notebook will:

- Create a specified amount of instances
- Send datasets to those instances
- Send processing jobs and parameters to instances
- Run jobs
- Monitor jobs
- Retrieve and store results
___
### Information on Provided Files ###
This repository contains the following folders:

   - examples
   - keys
   - documentation
   - images
   - movies
   - logs
   - figures
   - results

___
#### examples: ####
This folder provides example files that will allow you to understand how files are being fed to this notebook, and how they are formated, to help you understand how to generate your files to allow this notebook to run smoothly.

#### keys: ####
This folder should hold your ssh or .pem keys that you generated for your EC2 instance

#### documentation: ####
This should should hold the .csv that describes your EC2 instance names and attributes.

#### images: ####
This should hold any images generated.

#### movies: ####
This folder should hold any movies generated.

#### servers: ####
This folder should hold any logs generated relating to server performance, states, and program performance and states.

#### figures: ####
This folder should hold any figures generated.

#### results: ####
This folder should hold any results generated.

___
### Dependencies
This notebook requires the following dependenices

   - boto3
   - botocore
   - jmespath


___
### Setting up AWS Config

We will be using the Boto3 API to interface Python with AWS, before we can do that we need to install the AWS CLI (Command Line Interface) from here:

https://aws.amazon.com/cli/

After you have installed the CLI, you must configure your AWS credentials on the machine that you will run this code on, the configuration process can be found here:

https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#configuration

Essentially what you will have to do is in a terminal type:
    
    aws config

After which you will be asked for security credentials, these credentials will be avaliable on your AWS account under 
"My Security Credentials"

After this is done, you will be asked an avaliability zone, for now use: 

    us-east-2

Finally, for your output use: 
    
    json

<b>Ensure that in the configure step you set your region to the region that your EC2 and S3 instances and buckets are at, else you will not be able to access them with the API</b> 

___

### Viewing your current EC2 Instances###

Using the Boto3 library, we are able to acquire information about our instances that are in AWS using this Python API, below we are importing the libraries we will need:

In [1]:
import sys
import boto3
import botocore
from botocore.exceptions import ClientError
import time
import os
import platform

Now that the libraries are acquired, we will use the ec2 resource to see what instances we currently have on our console:

In [2]:
# creating ec2 resource, ec2 client, and s3 connection
try:
    ec2 = boto3.resource('ec2')
    client = boto3.client('ec2')
    s3 = boto3.resource('s3')
except:
    print("Error creating clients, check AWS configuration in AWS CLI")
    sys.exit()
    
# printing instances created 
print("EC Resource:", ec2)
print("EC Client:", client)
print("S3 Connection:", s3, "\n")

# printing what running instances we have up
print("EC2 Instances Running:")

# filtering for instances by name and if running
running_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])

# printing the names of running instances and their machine type
for instance in running_instances:
    print(instance.id, instance.instance_type)

# printing what stopped instances we have
print("\nEC2 Instances Stopped:")

# filtering for instances by name and if stopped
stopped_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['stopped']}])

# printing the names of running instances and their machine type
for instance in stopped_instances:
    print(instance.id, instance.instance_type)

# for other cases other than running or stopped
other_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['pending', 'shutting-down',
                                                    'terminated','stopping']}])

print("\nEC2 Instances Other:")
for instance in other_instances:
    print(instance.id, instance.instance_type)
    
print("\nDone Searching")

EC Resource: ec2.ServiceResource()
EC Client: <botocore.client.EC2 object at 0x7f1078204f28>
S3 Connection: s3.ServiceResource() 

EC2 Instances Running:

EC2 Instances Stopped:
i-0d606940d0ad53cd1 t2.micro
i-06413b62ffcd07c4d t2.micro
i-03a2ab946f0ac123e t2.micro

EC2 Instances Other:
i-0e00210454f976d3d t2.micro
i-092e862fbd7ee7817 t2.micro
i-04cdada4585644359 t2.micro
i-097d249c169d431a0 t2.micro
i-0046db8e0d2d8e080 t2.micro

Done Searching


### Setting up the instances needed ###

We will now be preparing the instances that we want to process the images with.

<b> Warning: in the following process we will be stopping all of the current instances and creating a defined amount of instances that we will use for our image processing </b>

You will be asked to confirm that you want to begin the process of pausing your instances and starting the server process.

In [3]:
# Waits for user to authorize stop of their instances
go = False
while(go == False):
    user = input("Authorize to stop and store current instances ([Y]/n): ")
    if (user == "Y"):
        go = True
    elif (user == 'n'):
        print("Entered No: Exiting Program")
        sys.exit()
    else:
        print("Invalid Input:", user)

# Filters a list of all instances before we create new ones
old_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 
              'Values': ['running', 'stopped',
                        'pending','shutting-down'
                         ,'stopping']
             }])

# Stores ID of old instances and then stops all instances
print("Stopping:")
stopIds = []
for instance in old_instances:
    instance.stop(instance.id)
    stopIds.append(instance.id)
print(stopIds)    

Authorize to stop and store current instances ([Y]/n): Y
Stopping:
['i-0d606940d0ad53cd1', 'i-06413b62ffcd07c4d', 'i-03a2ab946f0ac123e']


### Creating new instances ###

Now that the above instances are stopped, we want to create the number of instances that we want to use. AWS by default only allows you to run 20 instances in reserve, with a more limited number on On-Demand time. This notebook is set to the 20 instance reserved limit. 

You can request more instance allocations from a link on this page: https://aws.amazon.com/ec2/faqs/#How_many_instances_can_I_run_in_Amazon_EC2

Then change the 20 on line 6 to the maximum number of instances you can use.

Input the number of instances you want to use below:

In [4]:
answer = True
while answer:
    number_of_instances = abs(int(input("Number of instances to use: ")))
    if (number_of_instances <= 0):
        print("Invalid: Enter valid number")
    elif (number_of_instances > 20):
        print("Invalid: Cannot create more than 20 Instances")
    else:
        answer = False
        
print(number_of_instances, "EC2 instances will be created.")
number_of_instances

Number of instances to use: 3
3 EC2 instances will be created.


3

We will now create a key pair that will use to SSH into our servers. We will share the same key with every server for convenience.

In [5]:
# Creating key pair for Eclipse Megamovie usages
keyname = 'EMkey'
try:
    ec2.create_key_pair(KeyName=keyname)
except:
    print("Key already created or invalid.")
keyname

Key already created or invalid.


'EMkey'

We will create a list of names for our new EC2 Instances

In [6]:
base = "ECMEGA-SERVER-"
namelist = []
for name in range(number_of_instances):
    newname = base + str(name+1)
    namelist.append(newname)
namelist

['ECMEGA-SERVER-1', 'ECMEGA-SERVER-2', 'ECMEGA-SERVER-3']

We now want to find and get an AMI (Amazon Machine Image) that is an image of a server that is already set up to run the software that we want. There will be a public image for a MegaMovie configured server on AWS. We will find it by filtering AMI images by our owner id:

In [7]:
megaImage = ec2.images.filter(Filters=[{'Name':'owner-id', 'Values':['346926079389']}])
print("Images avaliable from Eclipse MegaMovie AWS:")
count = 1
avaliableImages = []
for images in megaImage:
    print(str(count)+".",images.name, images.id, images.architecture)
    count += 1
    avaliableImages.append(images)
avaliableImages

Images avaliable from Eclipse MegaMovie AWS:
1. Master-Image ami-027369ffdea1405c6 x86_64
2. Master-Image ami-0db7659b19b8677f5 x86_64


[ec2.Image(id='ami-027369ffdea1405c6'), ec2.Image(id='ami-0db7659b19b8677f5')]

Type the number printed above of the image of the server that you want to create the instances with below:

In [8]:
status = True 
length = len(avaliableImages)
while status:
    chooseImage = int(input("Enter number corresponding to image above: "))
    if chooseImage < 1: 
        print("Invalid number, enter a valid number")
    elif chooseImage > length:
        print("Invalid number, enter a valid number")
    else:
        print("Using image:", avaliableImages[chooseImage-1])
        status = False
usingImage = avaliableImages[chooseImage-1]
usingImage.name

Enter number corresponding to image above: 1
Using image: ec2.Image(id='ami-027369ffdea1405c6')


'Master-Image'

Finally, we want to choose a security group that we will assign to the servers. The security group defines who can access the server and how they can access it. There is a MegaMovie group avaliable that allows you to login and access the notebooks that are running on individual servers. We will now list the avaliable security groups below:

In [9]:
secureGroups = ec2.security_groups.all()
print("Avaliable Security Groups:")
val = 0
avaliableGroups = []
for groups in secureGroups:
    val += 1
    print(str(val) +".",groups.description, groups.id)
    avaliableGroups.append(groups)
avaliableGroups

Avaliable Security Groups:
1. launch-wizard-1 created 2018-11-14T11:04:00.006-08:00 sg-015d5ba79fb259b72
2. launch-wizard-7 created 2018-11-17T10:19:54.408-08:00 sg-023aed919f878ebbe
3. launch-wizard-2 created 2018-11-16T10:21:39.152-08:00 sg-03563349601f6c0a8
4. launch-wizard-4 created 2018-11-16T14:56:34.806-08:00 sg-03cd3639298cadbc7
5. launch-wizard-3 created 2018-11-16T14:45:07.432-08:00 sg-043d0b3e381abac9d
6. Security Group For Eclipse Megamovie sg-04aad0e0fdad5d76a
7. launch-wizard-6 created 2018-11-17T10:09:04.659-08:00 sg-050efbbde77ac2fb4
8. This security group was generated by AWS Marketplace and is based on recommended settings for Ubuntu Server 14.04 LTS (HVM) version 14.04 LTS 20180818 provided by Canonical Group Limited sg-0631d2aaba9aaa914
9. launch-wizard-8 created 2018-11-19T11:08:48.459-08:00 sg-072bb029b1e3a974e
10. Test security group for Jupyter access for Megamovie project sg-07dcb7d6071a2c647
11. launch-wizard-5 created 2018-11-16T16:39:49.516-08:00 sg-097b8f2d

[ec2.SecurityGroup(id='sg-015d5ba79fb259b72'),
 ec2.SecurityGroup(id='sg-023aed919f878ebbe'),
 ec2.SecurityGroup(id='sg-03563349601f6c0a8'),
 ec2.SecurityGroup(id='sg-03cd3639298cadbc7'),
 ec2.SecurityGroup(id='sg-043d0b3e381abac9d'),
 ec2.SecurityGroup(id='sg-04aad0e0fdad5d76a'),
 ec2.SecurityGroup(id='sg-050efbbde77ac2fb4'),
 ec2.SecurityGroup(id='sg-0631d2aaba9aaa914'),
 ec2.SecurityGroup(id='sg-072bb029b1e3a974e'),
 ec2.SecurityGroup(id='sg-07dcb7d6071a2c647'),
 ec2.SecurityGroup(id='sg-097b8f2d123cf007a'),
 ec2.SecurityGroup(id='sg-0fc4818efcc8ed55d'),
 ec2.SecurityGroup(id='sg-ca8a2aa6')]

Now enter a number to select the security group you would like to use:

In [10]:
status = True 
length = len(avaliableGroups)
while status:
    chooseGroup = int(input("Enter number corresponding to group above: "))
    if chooseGroup < 1: 
        print("Invalid number, enter a valid number")
    elif chooseGroup > length:
        print("Invalid number, enter a valid number")
    else:
        print("Using image:", avaliableGroups[chooseGroup-1])
        status = False
usingGroup = avaliableGroups[chooseGroup-1]
usingGroup.description

Enter number corresponding to group above: 6
Using image: ec2.SecurityGroup(id='sg-04aad0e0fdad5d76a')


'Security Group For Eclipse Megamovie'

We now have all the information we need to create the instances for our run, here are the settings that you chose to create the instances with:

In [11]:
print("- Eclipse Processing Server Information -\n")
print("Number of Instances:", number_of_instances)
print("SSH Keyname:", keyname)
print("Security group:", usingGroup.id)
print("AMI", usingImage.id)
print("\nServer Names:")
for name in namelist:
    print("\t"+str(name))

- Eclipse Processing Server Information -

Number of Instances: 3
SSH Keyname: EMkey
Security group: sg-04aad0e0fdad5d76a
AMI ami-027369ffdea1405c6

Server Names:
	ECMEGA-SERVER-1
	ECMEGA-SERVER-2
	ECMEGA-SERVER-3


#### Confirm below that you would like to create the instances above with the given parameters ##

In [12]:
leave = True
while leave:
    confirm = input("Confirm instance creation([Y]/n): ")
    if confirm == "Y":
        leave = False
    elif confirm == "n" or confirm == "N":
        print("Exiting program")
        sys.exit()
    else:
        print("Bad input")

# Creating the instances!
server_start = time.time()
created_instances = []

for i in range(number_of_instances):
    server_instance = ec2.create_instances(ImageId=usingImage.id, 
                                            MinCount=1,
                                            MaxCount=1, 
                                            Placement={'AvailabilityZone': 'us-east-2b',}, 
                                            InstanceType='t2.micro', 
                                            SecurityGroupIds=[usingGroup.id], 
                                            TagSpecifications=[{'ResourceType':'instance',
                                                                'Tags': [{'Key': 'Name',
                                                                          'Value': namelist[i]}
                                                                         ,]},])
    created_instances.append(server_instance)
created_instances

Confirm instance creation([Y]/n): Y


[[ec2.Instance(id='i-06a4cc1ccbd1f2952')],
 [ec2.Instance(id='i-06219c033c7247a3f')],
 [ec2.Instance(id='i-07e7b0079e3f5ef99')]]

### Uploading software and datasets to the new instances

Since we now have our servers up and running, we need to transfer our scripts and data so we can begin our processing tasks.

There is a public Eclipse Megamovie S3 bucket that will contain software for different processing tasks. You can choose to use the software that is there or you can choose to use software from the local machine.

If you choose to use use S3 you will:

- Get a list of all the software in the S3 bucket
- Choose which script you want to run
- Define a timeout scenario
- Authorize that run
    
If you choose to use software on local machine:

- All software in local "Software folder" will be copied to each instance
- You will define a "main script" eg. "Servertester.py", that will be run
- Define a timeout scenario
- Authorize the run
    
More information will follow on what will happen when the scripts run and output, for now we will begin copying our information.

#### For now, choose if you will use software on S3 or software from your local machine:

In [13]:
go = False
while not go:
    choice = input("Choose where to pull software from (s3/local): ")
    if choice == "s3":
        print("Using S3 software")
        go = True
    elif choice == "local":
        print("Using local software")
        go = True
    else:
        print("Invalid choice")

Choose where to pull software from (s3/local): local
Using local software


Below is listed software from both S3 and local machine to aid in choosing the main script that you want to deploy to the instances. Software on S3 is from the public bucket "Software", and local machine output is from current working directory "Software" folder, if it exists.

In [14]:
directory = os.getcwd()
exists = False

operating = platform.system()
print("**** LOCAL MACHINE ****\nOperating system:", operating)

for file in os.listdir(directory):
    if file == "Software" and operating == "Windows":
        exists = True
        softDir = str(directory) + "\\" + "Software"
    elif file == "Software" and operating == "Linux":
        exists = True
        softDir = str(directory) + "/" + "Software"
if exists:
    print("Software directory at:", softDir)
    filesInSoft  = []
    print("-------------------------------------")
    for software in os.listdir(softDir):
        filesInSoft.append(software)
        print(software)
    print("-------------------------------------\n"
          + "Files in software:", len(filesInSoft))
else:
    print("Software directory does not exist")

print("\n\n**** S3 ECLIPSE BUCKET ****")

**** LOCAL MACHINE ****
Operating system: Linux
Software directory at: /home/admin/MegaMovie/aws-deployment/eclipse-deployment-system/Software
-------------------------------------
ServerTesters.py
SingleProcessTest.py
-------------------------------------
Files in software: 2


**** S3 ECLIPSE BUCKET ****
